In [2]:
#shuffle
#if this works, try combinations with patsy
#move xgtest out
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_absolute_error
from random import randint

train = pd.read_csv('./data/allstate/train.csv')
test = pd.read_csv('./data/allstate/test.csv')
test['loss'] = np.nan
joined = pd.concat([train, test])

def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    con = 2
    x =preds-labels
    grad =con*x / (np.abs(x)+con)
    hess =con**2 / (np.abs(x)+con)**2
    return grad, hess 

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))


cat_feature = [n for n in joined.columns if n.startswith('cat')]    
cont_feature = [n for n in joined.columns if n.startswith('cont')] 
                          
for column in cat_feature:
    joined[column] = pd.factorize(joined[column].values, sort=True)[0]

train = joined[joined['loss'].notnull()]
test = joined[joined['loss'].isnull()]

shift = 140
y = np.log(train['loss'] + shift)
ids = test['id']
X = train.drop(['loss', 'id'], 1)
X_test = test.drop(['loss', 'id'], 1)

from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators = 200, random_state=0, verbose=True)
forest.fit(X,y)
importances = forest.feature_importances_
print("Feature ranking:")
names = x.columns
print (sorted(zip(map(lambda x: round(x, 4), importances), names), reverse=True))

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 18.2min finished


Feature ranking:
[(0.23730000000000001, 'cat80'), (0.053800000000000001, 'cont14'), (0.047199999999999999, 'cat101'), (0.043700000000000003, 'cont7'), (0.034200000000000001, 'cont2'), (0.029600000000000001, 'cat79'), (0.026700000000000002, 'cat103'), (0.0241, 'cat100'), (0.022599999999999999, 'cat111'), (0.020899999999999998, 'cat12'), (0.0207, 'cat112'), (0.018700000000000001, 'cont8'), (0.0184, 'cont5'), (0.017899999999999999, 'cont3'), (0.017100000000000001, 'cont6'), (0.017100000000000001, 'cat53'), (0.016799999999999999, 'cont4'), (0.0166, 'cat81'), (0.015800000000000002, 'cont1'), (0.0152, 'cat110'), (0.014800000000000001, 'cont13'), (0.013599999999999999, 'cont12'), (0.012999999999999999, 'cont11'), (0.012999999999999999, 'cont10'), (0.012200000000000001, 'cont9'), (0.012, 'cat57'), (0.012, 'cat114'), (0.011900000000000001, 'cat1'), (0.0109, 'cat113'), (0.0094000000000000004, 'cat72'), (0.0091999999999999998, 'cat116'), (0.0067999999999999996, 'cat83'), (0.0064000000000000003, '

In [27]:
sorted_features = sorted(zip(map(lambda x: round(x, 3), importances), names), reverse=True)
selected_columns = [x[1] for x in sorted_features if x[0] > 0.001]
print(list(selected_columns))

['cat80', 'cont14', 'cat101', 'cont7', 'cont2', 'cat79', 'cat103', 'cat100', 'cat111', 'cat12', 'cat112', 'cont8', 'cont5', 'cont3', 'cont6', 'cont4', 'cat81', 'cat53', 'cont1', 'cont13', 'cat110', 'cont12', 'cont11', 'cont10', 'cont9', 'cat57', 'cat114', 'cat1', 'cat113', 'cat72', 'cat116', 'cat83', 'cat91', 'cat82', 'cat107', 'cat105', 'cat93', 'cat115', 'cat84', 'cat73', 'cat109', 'cat108', 'cat106', 'cat97', 'cat95', 'cat94', 'cat92', 'cat87', 'cat75', 'cat5', 'cat4', 'cat104', 'cat99', 'cat96', 'cat9', 'cat6', 'cat50', 'cat49', 'cat44', 'cat39', 'cat38', 'cat37', 'cat36', 'cat31', 'cat27', 'cat26', 'cat23', 'cat2', 'cat102']


In [28]:
X = X[list(selected_columns)]
X_test = X_test[list(selected_columns)]

In [30]:
X_test.shape

(125546, 69)

In [31]:
#X = X.sample(frac=0.1)
#y = y .iloc[X.index.values]

n_folds  = 6
kf = KFold(X.shape[0], n_folds=n_folds)
prediction = np.zeros(ids.shape)

final_fold_prediction= []
final_fold_real = []

partial_evalutaion = open('temp_scores.txt','w')
for i, (train_index, test_index) in enumerate(kf):
    print('\n Fold %d' % (i + 1))
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]    

    RANDOM_STATE = randint(1,429496)
    params = {
        'min_child_weight': 1,
        'eta': 0.001,
        'colsample_bytree': 0.5,
        'max_depth': 12,
        'subsample': 0.8,
        'alpha': 1,
        'gamma': 1,
        'silent': 1,
        'verbose_eval': False,
        'seed': RANDOM_STATE
    }

    xgtrain = xgb.DMatrix(X_train, label=y_train)
    xgtrain_2 = xgb.DMatrix(X_val, label=y_val)

    xgtest = xgb.DMatrix(X_test)
    watchlist = [(xgtrain, 'train'), (xgtrain_2, 'eval')]                    

    model = xgb.train(params, xgtrain, 100000, watchlist, obj=logregobj, feval=evalerror, 
    early_stopping_rounds=300, verbose_eval=100)        
    prediction += np.exp(model.predict(xgtest)) - shift

    X_val = xgb.DMatrix(X_val) 
    temp_serises = pd.Series(np.exp(model.predict(X_val))-shift)
    final_fold_prediction.append( temp_serises )
    temp_serises = np.exp(y_val) -shift
    final_fold_real.append(temp_serises )

    temp_cv_score = mean_absolute_error(np.exp(model.predict(X_val))-shift, np.exp(y_val) -shift)

    partial_evalutaion.write('fold '+str(i)+' '+str(temp_cv_score)+'\n')
    partial_evalutaion.flush()

prediction = prediction/n_folds
submission = pd.DataFrame()
submission['id'] = ids    
submission['loss'] = prediction

submission.to_csv('./data/allstate/sub_xgboost_cv6_transformed.csv', index=False)

final_fold_prediction = pd.concat(final_fold_prediction,ignore_index=True)
final_fold_real = pd.concat(final_fold_real,ignore_index=True)

cv_score = mean_absolute_error(final_fold_prediction, final_fold_real)
print ("CV Score={}".format(cv_score))    


 Fold 1
[0]	train-mae:3175.39	eval-mae:3176.84
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 300 rounds.
[100]	train-mae:3159.02	eval-mae:3160.45
[200]	train-mae:3110.67	eval-mae:3111.97
[300]	train-mae:3026.44	eval-mae:3027.46
[400]	train-mae:2913.61	eval-mae:2914.21
[500]	train-mae:2782.29	eval-mae:2782.39
[600]	train-mae:2641.82	eval-mae:2641.5
[700]	train-mae:2498.9	eval-mae:2498.11
[800]	train-mae:2358.47	eval-mae:2357.51
[900]	train-mae:2223.96	eval-mae:2223.17
[1000]	train-mae:2097.8	eval-mae:2097.47
[1100]	train-mae:1981.2	eval-mae:1981.8
[1200]	train-mae:1874.89	eval-mae:1876.55
[1300]	train-mae:1778.91	eval-mae:1781.86
[1400]	train-mae:1693.1	eval-mae:1697.95
[1500]	train-mae:1616.89	eval-mae:1624.01
[1600]	train-mae:1549.61	eval-mae:1559.13
[1700]	train-mae:1490.51	eval-mae:1502.62
[1800]	train-mae:1438.87	eval-mae:1453.58
[1900]	train-mae:1393.84	eval-mae:1411.19
[2000]	train-mae:1354.62	ev